In [34]:
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly as py
import numpy as np

df = pd.read_csv('../isaFull.tsv', '\t')
#need to do this because Pandas and numpy don't consider NaN==NaN, only None=None
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['Visit date [EUPATH_0000091]'])
df['Visit date [EUPATH_0000091]'] = pd.to_datetime(df['Visit date [EUPATH_0000091]'])

## Create visit week column containing date on MONDAY of the week the visit took place in: -------------
df['Visit week'] = df['Visit date [EUPATH_0000091]'].apply(lambda d: d-dt.timedelta(days=d.weekday()))
#print(df)

## Group by participant then by visit week: -----------------------------------------------------------
index = pd.MultiIndex.from_frame(df[['Visit week', 'Participant_Id']])
df = df.set_index(index)
df['counts'] = df.groupby(level=['Visit week', 'Participant_Id']).size()
df = df.sort_index()

In [35]:
##2 visits in a week
df_2 = df[df['counts']==2]
df_2.index = range(len(df_2))


##True or False if the values in both visits are the same or not
##gives double the amount of weeks because each visit will be [0] and [1]. But for the % its irrelevant
other_array = []
for index, row in df_2.iterrows():
    data = df_2[ df_2['Visit week']==row['Visit week'] ]\
                [ df_2['Participant_Id']==row['Participant_Id'] ].reset_index(drop=True)
    array = []
    for column in df_2.columns:
        array.append(data.iloc[0][column] == data.iloc[1][column])
    other_array.append(array)
    
dataframe_true_false = pd.DataFrame(other_array, columns=df_2.columns)

##frequency Table for True False in all columns
dataframe_true_false_counts=dataframe_true_false.apply(pd.value_counts)
dataframe_true_false_counts=dataframe_true_false_counts.fillna(0)
dataframe_true_false_counts=dataframe_true_false_counts.T
#i dont understand why but I can only call the columns if I rename them...
dataframe_true_false_counts.columns=['False','True']
dataframe_true_false_counts['True_freq']=(dataframe_true_false_counts['True']/len(df_2))
dataframe_true_false_counts['False_freq']=(dataframe_true_false_counts['False']/len(df_2))

##bar chart
df_figure=dataframe_true_false_counts.drop(['counts','Participant_Id','Observation_Id','Visit week','Visit date [EUPATH_0000091]','Hospital admission date [EUPATH_0000319]','Hospital discharge date [EUPATH_0000320]','Days since enrollment [EUPATH_0000191]','Age at visit (years) [EUPATH_0000113]','Household_Id'])
dataframe_true_false_counts_figure = px.bar(df_figure,
            y=df_figure.index,
            x="True_freq",
            orientation='h',
            title="Frequency of equal values on both visits in a week")
dataframe_true_false_counts_figure.show()
#dataframe_true_false_counts_figure.write_html("Documents/HDI/synthetic-data-service/rebeca.html")
#dataframe_true_false_counts_figure.write_image("dataframe_true_false_counts_figure.pdf")
py.offline.plot(dataframe_true_false_counts_figure, filename='dataframe_true_false_counts_2weeks_figure.html')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



'dataframe_true_false_counts_2weeks_figure.html'

In [36]:
##3 weeks
##True or False if the values of all three visits are the same all the same or not
##gives triple the amount of weeks because each visit will be [0] and [1] and [2]. But for the % its irrelevant

df_3=df[df['counts']==3]
df_3.index=range(len(df_3))

other_array = []
for index, row in df_3.iterrows():
    data = df_3[ df_3['Visit week']==row['Visit week'] ]\
                [ df_3['Participant_Id']==row['Participant_Id'] ].reset_index(drop=True)
    array = []
    for column in df_3.columns:
            array.append(data.iloc[0][column] == data.iloc[1][column] == data.iloc[2][column] )
    other_array.append(array)
    
dataframe_true_false_3 = pd.DataFrame(other_array, columns=df_3.columns)

#frequency Table for True False in all columns
dataframe_true_false_counts_3=dataframe_true_false_3.apply(pd.value_counts)
dataframe_true_false_counts_3=dataframe_true_false_counts_3.fillna(0)
dataframe_true_false_counts_3=dataframe_true_false_counts_3.T
#i dont understand why but I can only call the columns if I rename them...
dataframe_true_false_counts_3.columns=['False','True']
dataframe_true_false_counts_3['True_freq']=(dataframe_true_false_counts_3['True']/len(df_3))
dataframe_true_false_counts_3['False_freq']=(dataframe_true_false_counts_3['False']/len(df_3))

#figure
df_figure_3=dataframe_true_false_counts_3.drop(['counts','Participant_Id','Observation_Id','Visit week','Visit date [EUPATH_0000091]','Hospital admission date [EUPATH_0000319]','Hospital discharge date [EUPATH_0000320]','Days since enrollment [EUPATH_0000191]','Age at visit (years) [EUPATH_0000113]','Household_Id'])
dataframe_true_false_counts_3_figure = px.bar(df_figure_3,
            y=df_figure_3.index,
            x="True_freq",
            orientation='h',
            title="Frequency of equal values on all 3 visits in a week")
dataframe_true_false_counts_3_figure.show()
py.offline.plot(dataframe_true_false_counts_3_figure, filename='dataframe_true_false_counts_3weeks_figure.html')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



'dataframe_true_false_counts_3weeks_figure.html'

In [32]:
dataframe_true_false_counts

,False,True,True_freq,False_freq
Observation_Id,1956.0,0.0,0.000000,1.000000
Participant_Id,0.0,1956.0,1.000000,0.000000
Household_Id,0.0,1956.0,1.000000,0.000000
Abdominal pain [HP_0002027],372.0,1584.0,0.809816,0.190184
Abdominal pain duration (days) [EUPATH_0000154],388.0,1568.0,0.801636,0.198364
Admitting hospital [EUPATH_0000318],18.0,1938.0,0.990798,0.009202
Age at visit (years) [EUPATH_0000113],1632.0,324.0,0.165644,0.834356
Anorexia [SYMP_0000523],186.0,1770.0,0.904908,0.095092
Anorexia duration (days) [EUPATH_0000155],194.0,1762.0,0.900818,0.099182
"Asexual Plasmodium parasite density, by microscopy [EUPATH_0000092]",1216.0,740.0,0.378323,0.621677


In [33]:
dataframe_true_false_counts_3

,False,True,True_freq,False_freq
Observation_Id,15.0,0.0,0.0,1.0
Participant_Id,0.0,15.0,1.0,0.0
Household_Id,0.0,15.0,1.0,0.0
Abdominal pain [HP_0002027],9.0,6.0,0.4,0.6
Abdominal pain duration (days) [EUPATH_0000154],9.0,6.0,0.4,0.6
Admitting hospital [EUPATH_0000318],0.0,15.0,1.0,0.0
Age at visit (years) [EUPATH_0000113],15.0,0.0,0.0,1.0
Anorexia [SYMP_0000523],6.0,9.0,0.6,0.4
Anorexia duration (days) [EUPATH_0000155],6.0,9.0,0.6,0.4
"Asexual Plasmodium parasite density, by microscopy [EUPATH_0000092]",12.0,3.0,0.2,0.8
